### Converting Original Dataset 

In [2]:
import os
import numpy as np
import h5py
from scipy.io import loadmat, savemat
from enum import Enum
from math import floor
from scipy.stats import zscore

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
def decode_utf16_array(array):
    """Decode MATLAB UTF-16 encoded uint16 arrays to strings."""
    array = array.flatten() if array.ndim == 2 else array
    return ''.join(chr(c) for c in array if c != 0)


def handle_cell_string_dataset(dataset, file_handle):
    """Handle MATLAB-style cell array of strings stored as references."""
    result = []
    for i in range(dataset.shape[0]):
        ref = dataset[i, 0]
        deref = file_handle[ref]
        value = deref[()]
        if isinstance(value, bytes):
            result.append(value.decode('utf-8'))
        elif isinstance(value, np.ndarray) and value.dtype == np.uint16:
            result.append(decode_utf16_array(value))
        else:
            result.append(value)
    return result

def iterate_group(data: h5py.Group, final_data: dict):
    """
    Recursively iterate through a h5py Group and print its structure.
    """
    for key in data.keys():
        item = data[key]
        if isinstance(item, h5py.Group):
            final_data[key] = {}
            iterate_group(item, final_data[key])
        else:
            final_data[key] = item[:]
    return final_data

def load_data_matfile(path: str, name: list[str]=None):
    data = None
    with h5py.File(path, 'r') as f:
        # List all groups in the file
        # print("Keys in the file:", list(f.keys()))
        # Access the dataset

        if len(name)==0:
            data = {}
            iterate_group(f, data)
            return data
        else:
            result = {}
            for key in name:
                if isinstance(f[key], h5py.Group):
                    # print("key is a group, iterating through it")
                    result[key] = iterate_group(f[key], {})
                elif key == 'FILE_ID':
                    result[key] = handle_cell_string_dataset(f[key], f)
                else:
                    # print("key is a dataset, returning data")
                    data = f[key][:]
                    data = data.T  # Transpose to match MATLAB's column-major order
                    result[key] = data 
            return result
        
def load_result_matfile(path: str):
    data = loadmat(path)
    return data


In [5]:
class SourceDataKeys(Enum):
    """
    Enum to represent different keys in the original mat file.
    """
    FILE_ID = 'FILE_ID'
    ANALYSIS_ID = 'analysis_ID'
    ANALYSIS_SCORE = 'analysis_SCORE'
    SFNC = 'sFNC'

### Downsampling

In [6]:
def convert_fnc_to_features(fnc_path: str, dest_path: str, name: str):
    file_path = os.path.join(fnc_path, f'{name}.mat')
    original_data = load_data_matfile(
        file_path,
        name=[
            SourceDataKeys.SFNC.value,
            SourceDataKeys.FILE_ID.value,
            SourceDataKeys.ANALYSIS_SCORE.value,
        ],
    )

    # --- find diagnosis column (case-insensitive) ---
    file_ids = original_data[SourceDataKeys.FILE_ID.value]
    label_index = next(
        (i for i, col in enumerate(file_ids) if "diagnosis" in col.lower()),
        None
    )
    if label_index is None:
        raise ValueError(f'No "diagnosis" column found in FILE_ID for {name}')

    labels = original_data[SourceDataKeys.ANALYSIS_SCORE.value][:, label_index]
    # Optional: match MATLAB’s strict check
    # uniq = np.unique(labels)
    # if not np.all(np.isin(uniq, [1, 2])):
    #     raise ValueError(f"Unexpected diagnosis codes: {uniq.tolist()}")
    labels = labels.reshape(-1, 1)

    fnc_matrices = original_data[SourceDataKeys.SFNC.value]  # shape: (N, P, P)
    N, P, _ = fnc_matrices.shape

    # --- build the lower-triangle mask excluding diagonal (k=-1) ---
    mask = np.tril(np.ones((P, P), dtype=bool), k=-1)  # P x P

    # --- replicate MATLAB's column-major flattening ---
    # 1) reshape each P×P to length P*P along Fortran (column-major) order
    # 2) take the linear indices where mask is True, also in Fortran order
    linear_idx = np.where(mask.ravel(order='F'))[0]           # size: P*(P-1)/2
    fnc_flat_F = fnc_matrices.reshape(N, P * P, order='F')    # N x (P*P)

    source_data = fnc_flat_F[:, linear_idx]                   # N x (P*(P-1)/2)
    # keep dtype default (float64) to match MATLAB; or cast if you prefer
    # source_data = source_data.astype(np.float64, copy=False)

    # append labels as the last column
    out = np.hstack([source_data, labels])

    # save with variable name = dataset name (like MATLAB)
    out_path = os.path.join(dest_path, f'{name}.mat')
    savemat(out_path, {name: out}, do_compression=True)


for i in ["FBIRN", "COBRE"]:
    fnc_dataset = convert_fnc_to_features("../original_dataset", "dataset", i)

In [10]:
### PARAMETERS
SamplingThs = 0.7
iter = 101
ntree = 201
NI_threshold = 2
TypThs = 0.8

In [8]:
import complete_random_forest.crf as crf

### Running CRF

In [9]:
from scipy.io import savemat


def perform_crf(
    dataset_path: str,
    result_path: str,
    name: str
):
    """
    consisting of subjects with there features as columns
    last column is label of the subject
    """

    file_path = os.path.join(dataset_path, f'{name}.mat')
    dataset = load_result_matfile(file_path)[name]

    subject_count, features_count = dataset.shape

    labels = dataset[:, -1]
    label_classes = np.unique(labels)
    len_label_classes = len(label_classes)
    # class is either 1 (SZ) or 2 (HC)
    original_cls_label_indexes = dict()
    sampling_cls_label_count = [None] * len_label_classes

    for i in range(len_label_classes):
        original_cls_label_indexes[label_classes[i]] = np.where(
            labels == label_classes[i]
        )[0]
        sampling_cls_label_count[i] = floor(
            len(original_cls_label_indexes[label_classes[i]]) * SamplingThs
        )

    dtype = np.dtype([
        ("subject_id", np.int64),   # col 0
        ("count", np.int64),        # col 1
        ("non_noise_count", np.int64),      # col 2
        ("ratio", np.float64)       # col 3
    ])

    sub_noise_per_iter = np.zeros(subject_count, dtype=dtype)
    sub_noise_per_iter["subject_id"] = np.arange(1, subject_count + 1, dtype=int) # id's of all subjects
    mean_sub_sampling_length = int(np.mean(sampling_cls_label_count))


    sampling_indexes = np.zeros(
        (len_label_classes * mean_sub_sampling_length, iter), dtype=int
    )

    non_noise_sampling_subjects = [] # for each sampling, subjects Ids which are not noise
    nltc_decisions = []
    for i in range(iter):
        print(f"Constructing No. {i+1} CRF for {name} dataset")
        index_temp = []
        for cls in label_classes:
            random_sampling_indexes = np.random.permutation(
                original_cls_label_indexes[cls]
            )[:mean_sub_sampling_length]
            index_temp.extend(random_sampling_indexes)

        sampling_indexes[:, i] = np.array(index_temp).astype(int)

        print("sampling indexes: ", sampling_indexes[:, i])

        sub_noise_per_iter['count'][sampling_indexes[:,i]] += 1
        sampled_dataset = dataset[sampling_indexes[:,i], :]
        attributes = zscore(sampled_dataset[:, 0:features_count-1], axis=0, ddof=1)
        sampled_dataset_labels = sampled_dataset[:, -1].reshape(-1,1)

        training_data = np.hstack((sampled_dataset_labels, attributes))

        # denoise_data, non_noise_ID, NLTC_labels =  running_crf(training_data, ntree, NI_threshold)
        instance = crf.CRF(ntree, NI_threshold)
        _, non_noise_ids, nltc_labels = instance.crf_v1(training_data, ntree)
        nltc_decisions.append(nltc_labels)
        print("picked sampling number: ", sampling_indexes[non_noise_ids, i])
        non_noise_sampling_subjects.append(sampling_indexes[non_noise_ids, i])

    denoise_check = np.zeros((subject_count,), dtype=int)
    for i in range(iter):
        idxs = non_noise_sampling_subjects[i]
        denoise_check [idxs] += 1

    # print("denoise count for each subject: ", denoise_check)
    sub_noise_per_iter['non_noise_count'] = denoise_check

    np.divide(sub_noise_per_iter['non_noise_count'], sub_noise_per_iter['count'], out=sub_noise_per_iter['ratio'], where=sub_noise_per_iter['count'] != 0)
    sub_noise_per_iter['ratio'] = np.round(sub_noise_per_iter['ratio'], 1)

    final_mat = np.column_stack((sub_noise_per_iter["subject_id"], sub_noise_per_iter["count"], sub_noise_per_iter["non_noise_count"], sub_noise_per_iter['ratio']))
    # print("iteration matrix: ", final_mat)

    output_path = os.path.join(result_path, f'{name}_CRF.mat')
    savemat(output_path, {
        'count': final_mat,
        'nltc_labels': nltc_decisions,
        'non_noise_ind': non_noise_sampling_subjects
    }, do_compression=True)


for name in ['FBIRN', 'COBRE']:
    perform_crf('dataset', 'results', name)


Constructing No. 1 CRF for FBIRN dataset
sampling indexes:  [253 181  17 127  86 138  44  63 278   3 205 255 101 153 131 185 217 139
 267 167 260  72  39 220   0 130  61 137 180 299 207 169  51  70   5 103
   2 102  18 246 184  88 224 227 228  23 173 147  96 266   1  77  98 305
  16  69 155 240  65 281 215 166 213 229 201 170 234 298 119 114  56  73
 194 252  64  46 243  78 307 128  99 226 259  79 241 210  24  41 290 107
 271 200 188 151 301 303 154 125 120  59 136 289 176  68  48 231 193 182
 157  58 223 164 236  14 192  50   6  52 263 238  81 291 242 195  26 277
 294 284 244 124 285 270 145 163  97 187 196 280 199 104 141 251 162 190
  83 189 216 309 208 152 160 179 118 287 177   4  34 110  12 221  95  42
 111 218 310 222 115 235  75  53  89 225 292 106  60 198 265  40  31  22
  11  10 171 168 135 219 245  47 257  36  82  67 140   8  55 212 158  84
 254  38 186 206 304  32 100 239 296  54 150 308 183 126 112 276 262  74]
picked sampling number:  [ 17 127  86 138  44  63   3 205 255 1

In [8]:
def pretty_print(mat):
    """Print with ints for first 3 cols, 1 decimal for last."""
    ids     = mat[:, 0].astype(np.int64)
    count   = mat[:, 1].astype(np.int64)
    success = mat[:, 2].astype(np.int64)
    ratio   = mat[:, 3]
    rows = [f"[{i:>3d} {c:>3d} {s:>3d} {r:>3.1f}]" for i,c,s,r in zip(ids, count, success, ratio)]
    print("\n".join(rows))

crf = load_result_matfile('results/FBIRN_CRF.mat')['count']
pretty_print(crf)

[  1  68  67 1.0]
[  2  76  76 1.0]
[  3  71  17 0.2]
[  4  75  75 1.0]
[  5  64   0 0.0]
[  6  69  66 1.0]
[  7  78  78 1.0]
[  8  73  72 1.0]
[  9  68  34 0.5]
[ 10  69  69 1.0]
[ 11  67  67 1.0]
[ 12  66  66 1.0]
[ 13  71   0 0.0]
[ 14  70   0 0.0]
[ 15  70  70 1.0]
[ 16  69  69 1.0]
[ 17  66   0 0.0]
[ 18  73   1 0.0]
[ 19  72  54 0.8]
[ 20  69  69 1.0]
[ 21  69  69 1.0]
[ 22  70  12 0.2]
[ 23  72  72 1.0]
[ 24  77  65 0.8]
[ 25  75  75 1.0]
[ 26  72  72 1.0]
[ 27  70  70 1.0]
[ 28  73  73 1.0]
[ 29  63  63 1.0]
[ 30  69   0 0.0]
[ 31  61   0 0.0]
[ 32  78   0 0.0]
[ 33  65   0 0.0]
[ 34  72  72 1.0]
[ 35  65   0 0.0]
[ 36  73   0 0.0]
[ 37  66  66 1.0]
[ 38  71   0 0.0]
[ 39  63  54 0.9]
[ 40  73  73 1.0]
[ 41  59   3 0.1]
[ 42  76  76 1.0]
[ 43  66  66 1.0]
[ 44  81  81 1.0]
[ 45  70  70 1.0]
[ 46  67  67 1.0]
[ 47  70  70 1.0]
[ 48  66  66 1.0]
[ 49  75  67 0.9]
[ 50  70  70 1.0]
[ 51  75   0 0.0]
[ 52  72  72 1.0]
[ 53  65  61 0.9]
[ 54  71   0 0.0]
[ 55  65  48 0.7]
[ 56  62  

In [9]:
def find_typical_subjects(input_path: str, data_path: str, name: str, typical_threshold: float) -> None:

    input_file = os.path.join(input_path, f'{name}_CRF.mat')
    crf_count = load_result_matfile(input_file)['count']
    crf_count[:, 3] = np.round(crf_count[:, 3], 1)

    data_file = os.path.join(data_path, f'{name}.mat')
    data = load_result_matfile(data_file)[name]

    original_labels = data[:, -1]
    
    mask_sz = (crf_count[:,3] >= typical_threshold) & (original_labels == 1)
    mask_hc = (crf_count[:,3] >= typical_threshold) & (original_labels == 2)

    typical_group_sz = np.where(mask_sz)[0]
    typical_group_hc = np.where(mask_hc)[0]

    savemat(f'results/{name}_Typ.mat', {
        'typical_sz': typical_group_sz,
        'typical_hc': typical_group_hc,
    }, do_compression=True)

for name in ["FBIRN", "COBRE"]:
    find_typical_subjects('results', 'dataset', name, TypThs)

### Finding Predictions

In [ ]:
from scipy import stats
from scipy.spatial.distance import cdist

# cummulative feature selection using Bonferroni corrected threshold 0.01/(Col-1)
def cumulative_features_selection(Pval, PvalPara):
    # Pval: 1-D array of p-values
    FeaInd = np.argsort(Pval)                  # indices sorted by p-value
    SortPval = Pval[FeaInd]                    # sorted p-values

    # Find first index where p-value exceeds threshold
    above_thresh = np.where(SortPval > PvalPara)[0]
    if above_thresh.size > 0:
        ind = above_thresh[0]                  # first index above threshold
    else:
        ind = len(SortPval)                    # all p-values below threshold

    Fea = FeaInd[:ind]                         # keep only significant features
    return Fea

def compute_score(independent_data, typical_data):
    col = independent_data.shape[1]
    
    ind_fea = independent_data[:, :-1]
    typical_data_features = typical_data[:, : -1]
    
    typical_data_labels = typical_data[:, -1]
    typical_group_sz = typical_data_features[typical_data_labels == 1, :]
    typical_group_hc = typical_data_features[typical_data_labels == 2, :]
    
    t_stat, p_val = stats.ttest_ind(typical_group_sz, typical_group_hc, axis=0, equal_var=True)
    
    # print('pvals: ', p_val)
    
    significant_threshold = 0.01 / (col - 1)
    
    selected_features = cumulative_features_selection(p_val, significant_threshold)
    # print('selected features: ', selected_features)
    
    center_sz = np.mean(typical_group_sz[:, selected_features], axis=0).reshape(1, -1)
    center_hc = np.mean(typical_group_hc[:, selected_features], axis=0).reshape(1, -1)
    
    
    # print(center_sz, center_hc)
    
    X = ind_fea[:, selected_features]
    dist1 = cdist(X, center_sz)
    dist2 = cdist(X, center_hc)
    
    distance_typical_group_sz = dist1.mean(axis=1)
    distance_typical_group_hc = dist2.mean(axis=1)
    
    total_distance = distance_typical_group_sz + distance_typical_group_hc
    
    A = distance_typical_group_sz / total_distance
    B = distance_typical_group_hc / total_distance
    
    scores = np.tan((A-B)*np.pi / 2)
    # print('final_scores: ', scores)
    
    return scores
    

def predict_score(subjects: list[str], input_path: str, dataset_path: str, typical_threshold: float):
    
    total_columns = len(subjects)+1
    all_cols = np.arange(total_columns)
    
    for subject_id in range(len(subjects)):
        
        sub_name = subjects[subject_id]
        
        data_file = os.path.join(dataset_path, f'{sub_name}.mat')
        independent_data = load_result_matfile(data_file)[sub_name]
        subject_count, _ = independent_data.shape
        
        ind_sub_labels = independent_data[:, -1]
        total_columns = len(subjects)+1
        independent_score = np.zeros((subject_count, total_columns))
        
        for main_sub_id in range(len(subjects)):
            if main_sub_id == subject_id:
                independent_score[:, subject_id] = ind_sub_labels
                continue
            
            main_sub_name = subjects[main_sub_id]
            main_sub_path = os.path.join(dataset_path, f'{main_sub_name}.mat')
            main_sub_data = load_result_matfile(main_sub_path)[main_sub_name]
            
            input_crf_path = os.path.join(input_path, f'{main_sub_name}_CRF.mat')
            main_sub_crf = load_result_matfile(input_crf_path)['count']
            
            typical_data = main_sub_data[main_sub_crf[:, 3] >= typical_threshold, :]

            independent_score[:, main_sub_id] = compute_score(independent_data, typical_data)
        independent_score[:, -1] = independent_score[:, np.setdiff1d(all_cols, [subject_id, total_columns-1])].mean(axis=1)
        idx = (independent_score[:, :len(subjects)] == 0).any(axis=1)
        independent_score[idx, -1] = 0
        
        output_path = os.path.join(input_path, f'{sub_name}_Score.mat')
        savemat(output_path, {
            sub_name: independent_score
        }, do_compression = True)
    
predict_score(['FBIRN', 'COBRE'], 'results/', 'dataset/', TypThs)

FBIRN :  [1]
COBRE :  [0]


In [15]:
DataName = ['FBIRN', 'COBRE']

def handle_typical_groups(name: str, group: str, data: h5py.Dataset):
    print(f"Dataset: {i}", f"Group: {group}")
    
    # single row, consisting of subject ID's 
    # representing typical subjects after all the iteration 
    print(f"shape: {data.shape} and type: {type(data)}")
    print("Total no.of Typical Subjects: ", data.shape[0])
    print(data)

for i in DataName:
    path= f'results/{i}_Typ.mat'
    data = load_result_matfile(path)
    
    handle_typical_groups(i, "SZ", data['typical_sz'])
    handle_typical_groups(i, "HC", data['typical_hc'])
    
    print('\n\n')

Dataset: FBIRN Group: SZ
shape: (1, 101) and type: <class 'numpy.ndarray'>
Total no.of Typical Subjects:  1
[[  0   1   3   5   7  18  20  23  24  27  39  41  43  44  46  48  51  56
   59  61  62  63  68  69  72  73  77  78  79  80  86  92  93  94  98  99
  101 107 113 119 120 121 125 127 130 132 138 139 142 147 153 154 159 161
  166 167 169 170 181 185 188 193 194 197 201 202 203 204 205 209 220 224
  228 229 231 233 234 240 241 243 246 250 252 255 259 260 267 272 275 279
  281 283 286 289 290 297 298 299 302 303 305]]
Dataset: FBIRN Group: HC
shape: (1, 95) and type: <class 'numpy.ndarray'>
Total no.of Typical Subjects:  1
[[  6   9  10  11  14  15  19  22  25  26  28  33  36  38  42  45  47  49
   52  55  57  58  71  75  81  84  85  87  95  97 100 105 106 110 117 118
  122 123 126 135 140 141 144 148 150 156 160 162 163 164 171 172 179 183
  187 190 191 206 208 211 212 216 218 219 222 225 232 235 236 237 238 239
  242 244 245 249 257 261 262 263 265 273 274 276 277 280 284 285 287 2

In [13]:
import numpy as np

def handle_Indep_Score(name: str, data: h5py.Dataset):
    print(f"======= Independent Score: {name} ========")
    
    # total subjects of current dataset name, then its classification comparing other dataset 
    print(f"Shape: {data.shape} and type: {type(data)}")
    print("top 5 subjects and there score comparing other dataset")
    print(data[:5][:])
    
    total_subjects = data.shape[0]
    avg_column = data[:, -1]
    sz_subjects_count = np.count_nonzero(avg_column >= 0)
    hc_subjects_count = np.count_nonzero(avg_column < 0)
    print("count of subjects having SZ: ", sz_subjects_count)
    print("count of subjects who are healthy: ", hc_subjects_count)
    

DataName = ['FBIRN', 'COBRE']
for i in DataName:
    path = f'results/{i}_Score.mat'
    data = load_result_matfile(path)[i]
    handle_Indep_Score(i, data)
    # print('\n\n')

======= Independent Score: FBIRN ========
Shape: (311, 3) and type: <class 'numpy.ndarray'>
top 5 subjects and there score comparing other dataset
[[ 1.         -0.33058244 -0.33058244]
 [ 1.         -0.47686564 -0.47686564]
 [ 1.         -0.0847972  -0.0847972 ]
 [ 1.         -0.36939584 -0.36939584]
 [ 2.         -0.42497186 -0.42497186]]
count of subjects having SZ:  167
count of subjects who are healthy:  144
======= Independent Score: COBRE ========
Shape: (157, 3) and type: <class 'numpy.ndarray'>
top 5 subjects and there score comparing other dataset
[[-0.21598691  1.         -0.21598691]
 [-0.45276381  1.         -0.45276381]
 [-0.23919379  1.         -0.23919379]
 [ 0.05065942  2.          0.05065942]
 [ 0.55720257  2.          0.55720257]]
count of subjects having SZ:  41
count of subjects who are healthy:  116
